In [ ]:
# Movie Search - Neptune
https://aws.amazon.com/blogs/database/make-relevant-movie-recommendations-using-amazon-neptune-amazon-neptune-machine-learning-and-amazon-opensearch-service/
https://github.com/aws-samples/amazon-neptune-ontology-example-blog.git



In [1]:
!pip install -q opensearch-py

In [ ]:
# Reference 
https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-clients.html#serverless-python

In [2]:
import boto3

REGION = boto3.Session().region_name

# Replace with your S3 bucket where you prepared the IMDB data for Neptune use
SOURCE_S3_PATH_NOSLASH=f"s3://ekg-da-poc"

# Replace with your Amazon OpenSearch Service domain endpoint
FTS_AOS_ENDPOINT="https://z9urozgne3v5p4ywe2fj.us-east-1.aoss.amazonaws.com"

VS_AOS_ENDPOINT="https://8kd01utb7q4h0s0lec3g.us-east-1.aoss.amazonaws.com"

# Replace with your IAM role for the GNN embedding pipeline.
SAGEMAKER_ROLE="arn:aws:iam::196856463470:role/SandboxServiceRole"

In [3]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestsAWSV4SignerAuth
from opensearchpy.helpers import bulk
import boto3

service = 'aoss'
region2 = 'us-east-1'

service = 'aoss'

credentials = boto3.Session().get_credentials()
auth2 = RequestsAWSV4SignerAuth(credentials, region2, service)

vs_aos_client = OpenSearch(
    hosts = [{'host': VS_AOS_ENDPOINT.split("//")[1], 'port': 443}],
    http_auth=auth2,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize=20,
)


In [4]:
vs_aos_client.indices.get_alias(index="*")

{'movie_sentence': {'aliases': {}},
 'movie_2': {'aliases': {}},
 'movie_4': {'aliases': {}},
 'movie': {'aliases': {}},
 'movie_3': {'aliases': {}},
 'movie_6': {'aliases': {}},
 'movie_5': {'aliases': {}},
 'movie_8': {'aliases': {}},
 'movie_7': {'aliases': {}},
 'python-test-index': {'aliases': {}},
 'books-index': {'aliases': {}},
 'movie_sentence_4': {'aliases': {}},
 '.opensearch_dashboards_1': {'aliases': {'.opensearch_dashboards': {}}},
 'movie_sentence_3': {'aliases': {}},
 'movie_gnn': {'aliases': {}},
 'movie_sentence_2': {'aliases': {}},
 'book_2': {'aliases': {}},
 'movie_sentence_8': {'aliases': {}},
 'movie_sentence_7': {'aliases': {}},
 'movie_sentence_6': {'aliases': {}},
 'movie_gnn_6': {'aliases': {}},
 'movie_gnn_4': {'aliases': {}},
 'movie_gnn_3': {'aliases': {}},
 'movie_gnn_2': {'aliases': {}},
 'movie_gnn_8': {'aliases': {}},
 'movie_gnn_7': {'aliases': {}}}

In [5]:
import boto3
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

service = 'aoss'
region = 'us-east-1'

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

fts_aos_client = OpenSearch(
    hosts = [{'host': FTS_AOS_ENDPOINT.split("//")[1], 'port': 443}],
    http_auth=auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize=20,
)

In [6]:
fts_aos_client.indices.get_alias(index="*")

{'movie_4': {'aliases': {}},
 '.opensearch_dashboards_1': {'aliases': {'.opensearch_dashboards': {}}},
 'movie_5': {'aliases': {}}}

In [ ]:
#Reference
https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-clients.html#serverless-python
https://opensearch.org/docs/latest/clients/python-low-level/

In [9]:
# create an index
index_name = 'books-index'
create_response = vs_aos_client.indices.create(
    index_name
)

print('\nCreating index:')
print(create_response)


RequestError: RequestError(400, 'resource_already_exists_exception', 'OpenSearch exception [type=resource_already_exists_exception, reason=index [books-index/Iyy3m5EBIRMOPYHx6vl7] already exists]- server : [envoy]')

In [8]:
book_index_2 = {
    "settings": {
        "number_of_replicas": 1,
        "number_of_shards": 1,
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings" : {
    "properties" : {
      "director" : {
        "type" :  "text"
      },
      "title" : {
        "type" :  "text"
      },
      "year" : {
        "type" :  "text"
      },
      "id" : {
        "type" :  "text"
      }
    }
  }
}

In [7]:
#vs_aos_client.indices.delete(index="book_2")
vs_aos_client.indices.create(index="book_2",body=book_index_2)

NameError: name 'book_index_2' is not defined

In [9]:
#fts_aos_client.indices.delete(index="book_2")
fts_aos_client.indices.create(index="book_2",body=book_index_2)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'book_2'}

In [10]:
vs_aos_client.indices.get(index="books-index")

{'book_2': {'aliases': {},
  'mappings': {'properties': {'director': {'type': 'text'},
    'id': {'type': 'text'},
    'title': {'type': 'text'},
    'year': {'type': 'text'}}},
  'settings': {'index': {'number_of_shards': '2',
    'provided_name': 'book_2',
    'creation_date': '1725401422772',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': 'LCfwuZEByQpUVyt3W_Fx',
    'version': {'created': '136327827'}}}}}

In [11]:
vs_aos_client.indices.get(index="book_2")

{'book_2': {'aliases': {},
  'mappings': {'properties': {'director': {'type': 'text'},
    'id': {'type': 'text'},
    'title': {'type': 'text'},
    'year': {'type': 'text'}}},
  'settings': {'index': {'number_of_shards': '2',
    'provided_name': 'book_2',
    'creation_date': '1725401422772',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': 'LCfwuZEByQpUVyt3W_Fx',
    'version': {'created': '136327827'}}}}}

In [12]:
fts_aos_client.indices.get(index="book_2")

{'book_2': {'aliases': {},
  'mappings': {'properties': {'director': {'type': 'text'},
    'id': {'type': 'text'},
    'title': {'type': 'text'},
    'year': {'type': 'text'}}},
  'settings': {'index': {'number_of_shards': '2',
    'provided_name': 'book_2',
    'creation_date': '1725537633958',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': 'MwEOwpEBM3RLgC09xmpf',
    'version': {'created': '136327827'}}}}}

In [20]:
# index a document
from opensearchpy.helpers import bulk
documents = []
document = {
    "_op_type": "index",
    "_index": "book_2",
    "director": "Stephen King",
    "title": "The Green Mile",
    "year": "1996",
    "id": "1"
}
params = []
headers = []
documents.append(document)

print('\nIndexing - index1:')

#index_response1 = vs_aos_client.bulk(
#    index = "book_2",
#    body = documents,
#    id = "1"
#)
#print(index_response1)
bulk(vs_aos_client, documents, max_chunk_bytes=(100 * 1024))


Indexing - index1:


(1, [])

In [13]:
# index a document
from opensearchpy.helpers import bulk
documents = []
document = {
    "_op_type": "index",
    "_index": "book_2",
    "director": "Stephen King",
    "title": "The Green Mile",
    "year": "1996",
    "id": "1"
}
params = []
headers = []
documents.append(document)

print('\nIndexing - index1:')

#index_response1 = vs_aos_client.bulk(
#    index = "book_2",
#    body = documents,
#    id = "1"
#)
#print(index_response1)
bulk(fts_aos_client, documents, max_chunk_bytes=(100 * 1024))


Indexing - index1:


(1, [])

In [14]:
document = {
    "director": "Roland Emerich",
    "title": "The Independece Day",
    "year": "1996",
   "id": "4"
}
#vs_aos_client.index(index="book_2",body=document,op_type="index")
                    

In [16]:
fts_aos_client.index(index="book_2",body=document,op_type="index")

{'_index': 'book_2',
 '_id': 'TosRwpEB2m9k_8Pm-ag3',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 0}

In [26]:
document = {
    "director": "Steven Spielberg",
    "title": "Jaws",
    "year": "1996",
   "id": "tt000123456"
}
#vs_aos_client.index
#vs_aos_client.index(index="book_2",body=document,op_type="index")

In [27]:
fts_aos_client.index(index="book_2",body=document,op_type="index")

{'_index': 'book_2',
 '_id': 'T4sWwpEB2m9k_8Pmjagu',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 0}

In [21]:
q1 = "The Independence Day"
query1 = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q1,
      'fields': ['title^2', 'director']
    }
  }
}

print('\nSearching - Vector index1:' + q1)
search_response1 = vs_aos_client.search(
    body = query1,
    index = 'book_2'
)
print(search_response1)


Searching - Vector index1:The Independence Day
{'took': 52, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.6748097, 'hits': [{'_index': 'book_2', '_id': '1%3A0%3AbuYPupEBKwGr2Tq_gs1k', '_score': 1.6748097, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': '1%3A0%3ATYsRwpEB2m9k_8PmY6h4', '_score': 1.6748097, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': '1%3A0%3AyUsQupEBSE2l0lVlSg2V', '_score': 1.3862942, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}]}}


In [22]:
print('\nSearching - Full Text index1:' + q1)
search_response1 = fts_aos_client.search(
    body = query1,
    index = 'book_2'
)
print(search_response1)


Searching - Full Text index1:The Independence Day
{'took': 277, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.3862942, 'hits': [{'_index': 'book_2', '_id': 'TosRwpEB2m9k_8Pm-ag3', '_score': 1.3862942, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': 'jXASwpEB6eUa2ZVDXMLq', '_score': 0.5753642, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}]}}


In [28]:
q2 = "Jaws"
query2 = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q2,
      'fields': ['title^2', 'director']
    }
  }
}

print('\nSearching - Vector index1:' + q2)
search_response2 = vs_aos_client.search(
    body = query2,
    index = 'book_2'
)
print(search_response2)


Searching - Vector index1:Jaws
{'took': 33, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 3.3887196, 'hits': [{'_index': 'book_2', '_id': '1%3A0%3Ab-YSupEBKwGr2Tq_AM15', '_score': 3.3887196, '_source': {'director': 'Steven Spielberg', 'title': 'Jaws', 'year': '1996', 'id': 'tt000123456'}}]}}


In [29]:
print('\nSearching - Full Text index1:' + q2)
search_response1 = fts_aos_client.search(
    body = query2,
    index = 'book_2'
)
print(search_response1)


Searching - Full Text index1:Jaws
{'took': 44, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.605183, 'hits': [{'_index': 'book_2', '_id': 'T4sWwpEB2m9k_8Pmjagu', '_score': 1.605183, '_source': {'director': 'Steven Spielberg', 'title': 'Jaws', 'year': '1996', 'id': 'tt000123456'}}]}}


In [31]:
q3 = "Mile"
query3 = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q3,
      'fields': ['title^2', 'director']
    }
  }
}

print('\nSearching - Vector index1:' + q3)
search_response3 = vs_aos_client.search(
    body = query3,
    index = 'book_2'
)
print(search_response3)


Searching - Vector index1:Mile
{'took': 42, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 2.6520412, 'hits': [{'_index': 'book_2', '_id': '1%3A0%3Axkv_uZEBSE2l0lVlzQ0D', '_score': 2.6520412, '_source': {'director': 'Stephen King', 'title': 'The Green Mile', 'year': '1996', 'id': '1'}}]}}


In [32]:
print('\nSearching - FTS index1:' + q3)
search_response3 = fts_aos_client.search(
    body = query3,
    index = 'book_2'
)
print(search_response3)


Searching - FTS index1:Mile
{'took': 43, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.3862942, 'hits': [{'_index': 'book_2', '_id': 'jHAQwpEB6eUa2ZVDTMKf', '_score': 1.3862942, '_source': {'director': 'Stephen King', 'title': 'The Green Mile', 'year': '1996', 'id': '1'}}]}}


In [33]:
document = {
    "director": "Roland Emerich",
    "title": "The Independece Day: Resurgence",
    "year": "20"
}
vs_aos_client.index(index="book_2",id="5",body=document,op_type="index")

RequestError: RequestError(400, 'illegal_argument_exception', 'Document ID is not supported in create/index operation request')

In [37]:
document = {
    "director": "Roland Emerich",
    "title": "The Independece Day: Resurgence",
    "year": "2016",
    "id": "6"
}
fts_aos_client.index(index="book_2",id="6",body=document,op_type="index")

{'_index': 'book_2',
 '_id': '6',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 0}

In [40]:
document = {
    "director": "Tony Scott",
    "title": "Top Gun",
    "year": "1986",
    "id": "7"
}
fts_aos_client.index(index="book_2",id="7",body=document,op_type="index", refresh=True)

RequestError: RequestError(400, 'status_exception', 'true refresh policy is not supported.')

In [35]:
q1 = "The Independence Day"
query1 = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q1,
      'fields': ['title^2', 'director']
    }
  }
}

print('\nSearching - Vector index1:' + q1)
search_response1 = vs_aos_client.search(
    body = query1,
    index = 'book_2'
)
print(search_response1)


Searching - Vector index1:The Independence Day
{'took': 45, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.6748097, 'hits': [{'_index': 'book_2', '_id': '1%3A0%3AbuYPupEBKwGr2Tq_gs1k', '_score': 1.6748097, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': '1%3A0%3ATYsRwpEB2m9k_8PmY6h4', '_score': 1.6748097, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': '1%3A0%3AyUsQupEBSE2l0lVlSg2V', '_score': 1.3862942, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}]}}


In [38]:
print('\nSearching - FTS index1:' + q1)
search_response1 = fts_aos_client.search(
    body = query1,
    index = 'book_2'
)
print(search_response1)


Searching - FTS index1:The Independence Day
{'took': 38, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4, 'relation': 'eq'}, 'max_score': 1.3862942, 'hits': [{'_index': 'book_2', '_id': 'TosRwpEB2m9k_8Pm-ag3', '_score': 1.3862942, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': 'jXASwpEB6eUa2ZVDXMLq', '_score': 0.9400072, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': '5', '_score': 0.78038335, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day: Resurgence', 'year': '20'}}, {'_index': 'book_2', '_id': '6', '_score': 0.5753642, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day: Resurgence', 'year': '20', 'id': '6'}}]}}


In [44]:
def get_field(h, fname):
    if fname in h['_source']:
        return h['_source'][fname]
    else:
        return ""
    

In [58]:
q1 = "4"
query1 = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q1,
      'fields': ['id']
    }
  }
}

print('\nSearching ID - Vector index1:' + q1)
search_response1 = fts_aos_client.search(
    body = query1,
    index = 'book_2'
)
print(search_response1)
print("\n\nid|score|ID|director|title|year")
for idx, h in enumerate(search_response1["hits"]["hits"]):
    print(f"{h['_id']}|{h['_score']}|{get_field(h, 'id')}|{get_field(h, 'director')}|{get_field(h, 'title')}|{get_field(h, 'year')}")


Searching ID - Vector index1:4
{'took': 53, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.6931471, 'hits': [{'_index': 'book_2', '_id': 'jXASwpEB6eUa2ZVDXMLq', '_score': 0.6931471, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': 'TosRwpEB2m9k_8Pm-ag3', '_score': 0.6931471, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}]}}


id|score|ID|director|title|year
jXASwpEB6eUa2ZVDXMLq|0.6931471|4|Roland Emerich|The Independece Day|1996
TosRwpEB2m9k_8Pm-ag3|0.6931471|4|Roland Emerich|The Independece Day|1996


In [60]:
print(search_response1)
print("\n\nid|score|ID|director|title|year")
counter=0
id_toDelete=[]
for idx, h in enumerate(search_response1["hits"]["hits"]):
    print(f"{h['_id']}|{h['_score']}|{get_field(h, 'id')}|{get_field(h, 'director')}|{get_field(h, 'title')}|{get_field(h, 'year')}")
    counter = counter + 1
    id_toDelete.append({h['_id']})
if (counter>1):
    print(f"found {(counter -1)} duplicates for id={q1}")
iter = 1;
while (counter>1):
    print(f"deleting duplicates : in iteration=|{iter} : deleting {id_toDelete[iter]}")
    fts_aos_client.delete(index = "book_2",id = id_toDelete[iter])
    counter = counter - 1
    iter = iter +1
    
    
print('\nSearching After Removing Duplicates : ID - Vector index1:' + q1)
search_response1 = fts_aos_client.search(
    body = query1,
    index = 'book_2'
)
    
print(search_response1)
print("\n\nid|score|ID|director|title|year")
for idx, h in enumerate(search_response1["hits"]["hits"]):
    print(f"{h['_id']}|{h['_score']}|{get_field(h, 'id')}|{get_field(h, 'director')}|{get_field(h, 'title')}|{get_field(h, 'year')}")
    
                       

{'took': 53, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.6931471, 'hits': [{'_index': 'book_2', '_id': 'jXASwpEB6eUa2ZVDXMLq', '_score': 0.6931471, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}, {'_index': 'book_2', '_id': 'TosRwpEB2m9k_8Pm-ag3', '_score': 0.6931471, '_source': {'director': 'Roland Emerich', 'title': 'The Independece Day', 'year': '1996', 'id': '4'}}]}}


id|score|ID|director|title|year
jXASwpEB6eUa2ZVDXMLq|0.6931471|4|Roland Emerich|The Independece Day|1996
TosRwpEB2m9k_8Pm-ag3|0.6931471|4|Roland Emerich|The Independece Day|1996
found 1 duplicates for id=4
deleting duplicates : in iteration=|1 : deleting {'TosRwpEB2m9k_8Pm-ag3'}


NotFoundError: NotFoundError(404, '{"_index":"book_2","_id":"{\'TosRwpEB2m9k_8Pm-ag3\'}","_version":2,"result":"not_found","_shards":{"total":0,"successful":0,"failed":0},"_seq_no":0,"_primary_term":0}')

In [ ]:
# delete the index
#delete_response = vs_aos_client.indices.delete(
#    index_name
#)

#print('\nDeleting index:')
#print(delete_response)

In [17]:
index_name2 = 'python-test-index'
index_body2 = {
  'settings': {
    'index': {
      'number_of_shards': 4
    }
  }
}

print('\nCreating index2:')

create_response2 = vs_aos_client.indices.create(index_name2, body=index_body2)
print(create_response2)

vs_aos_client.indices.get(index="python-test-index")




Creating index2:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


{'python-test-index': {'aliases': {},
  'mappings': {},
  'settings': {'index': {'creation_date': '1724894679298',
    'number_of_shards': '2',
    'number_of_replicas': '0',
    'uuid': 'Jiy8m5EBIRMOPYHxEPmW',
    'version': {'created': '135217827'},
    'provided_name': 'python-test-index'}}}}

In [22]:
document2 = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}

print('\nIndexing - index1:')

index_response2 = vs_aos_client.index(
    index = 'python-test-index',
    body = document2
    #id = '1',
    #refresh = True
)


print(index_response2)


Indexing - index1:
{'_index': 'python-test-index', '_id': '1%3A0%3Azcy9m5EBKwGr2Tq_ha-l', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}


In [23]:
q2 = 'Miller'
query2 = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q2,
      'fields': ['title^2', 'director']
    }
  }
}

print('\nSearching - index2:')

search_response2 = vs_aos_client.search(
    body = query2,
    index = 'python-test-index'
)
print(search_response2)


Searching - index2:
{'took': 63, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.2876821, 'hits': [{'_index': 'python-test-index', '_id': '1%3A0%3AUjC8m5EBSE2l0lVlv8vF', '_score': 0.2876821, '_source': {'title': 'Moneyball', 'director': 'Bennett Miller', 'year': '2011'}}]}}


In [ ]:
# delete the index
#delete_response2 = vs_aos_client.indices.delete(
#    index_name2
#)

#print('\nDeleting index2:')
#print(delete_response2)